In [1]:
import pymp
from multiprocessing import Manager, cpu_count 
import sys
import os
import glob 
import gezi
from gezi import tqdm
import pandas as pd
import json
import tensorflow as tf
from absl import flags
FLAGS = flags.FLAGS
import melt

from plotly.graph_objs import Scatter,Layout
import plotly
import plotly.offline as py
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots

#setting offilne
plotly.offline.init_notebook_mode(connected=True)
tf.enable_eager_execution()

from projects.feed.rank.src import config
from projects.feed.rank.src.model import Model
from projects.feed.rank.src.tfrecord_dataset import Dataset

In [2]:
%%html
<style>
.output_wrapper, .output {
    height:auto !important;
    max-height:10000px;  /* your desired max-height here */
}
.output_scroll {
    box-shadow:none !important;
    webkit-box-shadow:none !important;
}
</style>

In [3]:
product='sgsapp'
mark='tuwen'

In [4]:
# 载入模型
model_hour = '2020050100'
valid_hour = '2020050103'
# valid_hour = '2020030812'
model_root = f'/search/odin/publicData/CloudS/libowei/rank_online/data/{mark}_hour_sgsapp_v1/models/8'
# model_root = '/home/gezi/tmp/rank/exps/tuwen/v29/dlrm-norm/'
model_dir = f'{model_root}/{model_hour}'
# model_dir = model_root
tfrecord_root = f'/search/odin/publicData/CloudS/libowei/rank/{product}/data/{mark}_hour_{product}_v1/tfrecords'
tfrecord_dir = f'{tfrecord_root}/{valid_hour}'
tfrecord_dir2 = f'{tfrecord_root}/{model_hour}'
print('model:', model_dir)
print('record:', tfrecord_dir)
argv = open(f'{model_dir}/command.txt').readline().strip().replace('data_version=2', 'data_version=1').split()
FLAGS(argv)
FLAGS.eager = True
# FLAGS.fields_norm=True
config.init()
model = Model()
melt.eager.restore(model, model_dir)

model: /search/odin/publicData/CloudS/libowei/rank_online/data/tuwen_hour_sgsapp_v1/models/8/2020050100
record: /search/odin/publicData/CloudS/libowei/rank/sgsapp/data/tuwen_hour_sgsapp_v1/tfrecords/2020050103


2020-05-23 23:04:36 0:00:00 Tf dataset and Tf model train in Eager mode, keras False, distributed:False
2020-05-23 23:04:36 0:00:00 log_level: 20 (try --debug to show more or --log_level=(> 20) to show less(no INFO), try --verbose to show train/valid loss intervaly)
2020-05-23 23:04:36 0:00:00 batch_size: 512 eval_batch_size: 512 batch_size_per_gpu: 512 num_gpus: 0 gpu: [] CUDA_VISIABLE_DEVICES=[-1] work_mode: train distributed: False horovod: False
2020-05-23 23:04:36 0:00:00 model: [8] model_dir: [/search/odin/libowei/rank/tuwen/model/8]
2020-05-23 23:04:36 0:00:00 Latest checkpoint: /search/odin/publicData/CloudS/libowei/rank_online/data/tuwen_hour_sgsapp_v1/models/8/2020050100/model.ckpt-2020050100-1.00-7957


In [5]:
dataset = Dataset('valid')

In [6]:
files = gezi.list_files(tfrecord_dir)
total = melt.get_num_records(files) 
total

575763

In [7]:
batch_size = 512
batches = dataset.make_batch(batch_size=batch_size, filenames=files, repeat=False)
num_steps = -int(-total // batch_size)

In [8]:
def gen_feats(batches, num_steps, ofile):
  with open(ofile, 'w') as out:
    print('mid,impression_time,docid,position,dur,click_feat,dur_feat', file=out)
    for x, _ in tqdm(batches, total=num_steps):
      mids = gezi.decode(x['mid'].numpy())
      docids = gezi.decode(x['docid'].numpy())
      positions = x['position'].numpy()
      durs = gezi.squeeze(x['duration'].numpy())
      impression_times = gezi.squeeze(x['impression_time'].numpy())
      model(x)
      click_feats = model.deep.click_feat.numpy()
      dur_feats = model.deep.dur_feat.numpy()
      for mid, impression_time, docid, position, dur, click_feat, dur_feat in zip(mids, impression_times, docids, positions, durs, click_feats, dur_feats):
        print(mid, impression_time, docid, position, dur, ' '.join(map(str, click_feat)), ' '.join(map(str, dur_feat)), sep=',', file=out)

In [9]:
gen_feats(batches, num_steps, f'/home/gezi/tmp/rerank/feats/valid/{valid_hour}.txt')

100%|██████████| 1125/1125 [02:41<00:00,  6.98it/s]


In [10]:
files = gezi.list_files(tfrecord_dir2)
total = melt.get_num_records(files) 
batches = dataset.make_batch(batch_size=batch_size, filenames=files, repeat=False)
num_steps = -int(-total // batch_size)
gen_feats(batches, num_steps, f'/home/gezi/tmp/rerank/feats/train/{model_hour}.txt')

100%|██████████| 3073/3073 [06:58<00:00,  7.35it/s]    


In [11]:
train_df = pd.read_csv(f'/home/gezi/tmp/rerank/feats/train/{model_hour}.txt')

In [13]:
len(train_df)

1573000

In [14]:
train_df['impression'] = train_df['mid'] + '\t' + train_df['impression_time'].astype(str)

In [15]:
train_df = train_df.sort_values(['impression', 'position'])

In [16]:
mid = train_df.mid.sample(1).values[0]

In [17]:
train_df[train_df.mid==mid]

,mid,impression_time,docid,position,dur,click_feat,dur_feat,impression
475846,ba96865937033144210,1588262505,20200425A0GETB00,0,0,-2.367145 -1.6647412 -3.3636727 -6.0215836 -1....,-5.2149854 -3.6094081 -4.545166 -4.0723243 -2....,ba96865937033144210\t1588262505
402207,ba96865937033144210,1588262505,20428k0l3vmxG1,1,0,-2.0335872 -1.4561865 -2.9174113 -5.889706 -1....,-4.3894453 -7.1682467 -8.394838 -7.707675 -2.5...,ba96865937033144210\t1588262505
1024317,ba96865937033144210,1588262505,20423i0l1ZgZTU,2,0,-1.9373739 -0.98902106 -1.6917835 -1.868069 -1...,-1.7407173 -12.77156 -15.076688 -13.543906 -1....,ba96865937033144210\t1588262505
251182,ba96865937033144210,1588263617,20200423A0U5M100,0,0,-1.7511288 -1.3083367 -2.6993248 -3.620738 -1....,-4.4357567 -4.9165683 -5.533539 -5.336941 -2.1...,ba96865937033144210\t1588263617
1279932,ba96865937033144210,1588263617,open_doc_prod27098762,1,23,-2.151579 -1.2168039 -2.80949 -3.3796303 -1.42...,-4.4778037 -5.8544774 -6.8430624 -6.3423276 -2...,ba96865937033144210\t1588263617
660139,ba96865937033144210,1588263617,20428n0l2Kc0I5,2,0,-3.0969675 -1.4869043 -2.2352421 -0.39854637 -...,-2.2273054 -26.101847 -31.27649 -27.670265 -1....,ba96865937033144210\t1588263617
671981,ba96865937033144210,1588263617,open_doc_prod27441970,3,0,-1.3036186 -0.8427943 -1.8812513 -2.4491687 -1...,-2.9650888 -9.718634 -11.141988 -10.240099 -1....,ba96865937033144210\t1588263617
775179,ba96865937033144210,1588263617,open_doc_prod27630370,4,0,-1.8898225 -0.9571392 -1.9957663 -0.43434992 -...,-3.008373 -11.336805 -13.45088 -12.048143 -1.9...,ba96865937033144210\t1588263617
1460928,ba96865937033144210,1588263617,20430f0l2GGiMK,5,0,-2.3241937 -0.9380552 -2.0523584 96.97018 -1.6...,-2.9368663 -13.868858 -16.570797 -14.715276 -1...,ba96865937033144210\t1588263617
693218,ba96865937033144210,1588263617,20430d0l31yrDB,6,0,-3.694609 -1.50496 -2.4728296 277.35797 -2.249...,-3.7510228 -23.945822 -29.135117 -25.503496 -2...,ba96865937033144210\t1588263617


In [18]:
impressions = train_df.impression.values

In [19]:
def gen_records(df, ofile):
  with melt.tfrecords.Writer(ofile) as writer:
    pre_impression = None
    feature = {}
    click_feats = []
    dur_feats = []
    positions = []
    durs = []
    mids = []
    num_records = 0
    for i in tqdm(range(len(df))):
      row = df.iloc[i]
      impression = row['impression']
      if impression != pre_impression:
        if durs:
          feature['durs'] = melt.gen_feature(durs, np.int64)
          feature['positions'] = melt.gen_feature(positions, np.int64)
          feature['impression'] = melt.gen_feature(row['impression'], df['impression'].dtype)
          feature['click_feats'] = melt.gen_feature(click_feats, np.float32)
          feature['dur_feats'] = melt.gen_feature(dur_feats, np.float32)
          feature['mids'] = melt.gen_feature(mids, df['mid'],dtype)
          record = tf.train.Example(features=tf.train.Features(feature=feature))
          writer.write(record)
          num_records += 1
        feature = {}
        click_feats = []
        dur_feats = []
        positions = []
        durs = []
        mids = []
        pre_impression = impression
      else:  
        click_feats += list(map(float, row['click_feat'].split()))
        dur_feats += list(map(float, row['dur_feat'].split()))
        positions += [row['position']]
        durs += [row['dur']]
        mids += [row['mid']]
        
    if durs:
      feature['dur'] = melt.gen_feature(durs, np.int64)
      feature['position'] = melt.gen_feature(positions, np.int64)
      feature['impression'] = melt.gen_feature(row['impression'], df['impression'].dtype)
      feature['click_feats'] = melt.gen_feature(click_feats, np.float32)
      feature['dur_feats'] = melt.gen_feature(dur_feats, np.float32)
      feature['mids'] = melt.gen_feature(mids, df['mid'],dtype)
      record = tf.train.Example(features=tf.train.Features(feature=feature))
      writer.write(record)
      num_records += 1
      
    with open(os.path.join(os.path.dirname(ofile), 'num_records.txt'), 'w') as out:
      print(num_records, file=out)

In [20]:
gen_records(train_df, '/home/gezi/tmp/rerank/tfrecords/train/2020050100/0.record')

  0%|          | 0/1573000 [00:00<?, ?it/s]


NameError: name 'dtype' is not defined

In [ ]:
valid_df = pd.read_csv(f'/home/gezi/tmp/rerank/feats/valid/{valid_hour}.txt')
valid_df['impression'] = valid_df['mid'] + '\t' + valid_df['impression_time'].astype(str)
valid_df = valid_df.sort_values(['impression', 'position'])

In [ ]:
gen_records(valid_df, '/home/gezi/tmp/rerank/tfrecords/valid/2020050103/0.record')